In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import math
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date


C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#read in the data
df= pd.read_csv("pressure logs", sep='\t', index_col=False)
df.drop(columns=["I_emission","I_grid","T-platinum","DUAL insulation"], inplace=True)
#df.drop(columns=['Threshold'], inplace=True)
#df.drop(labels=[0,1], axis=0)
df.head()


,Live comments,Time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,injection 1mbar,helium,T-CERNOX
0,NaN,25/10/2022 16:22:05,-0.002927,7.960000e-10,4.000000e-10,4.996000e-09,0.001554,304.0,4271.476
1,NaN,25/10/2022 16:22:13,-0.002706,7.960000e-10,4.000000e-10,4.996000e-09,0.001535,303.0,4271.363
2,NaN,25/10/2022 16:22:20,-0.003150,7.930000e-10,4.000000e-10,4.996000e-09,0.001524,304.0,4271.588
3,NaN,25/10/2022 16:22:27,-0.002977,7.900000e-10,3.990000e-10,4.996000e-09,0.001534,304.0,4271.251
4,NaN,25/10/2022 16:22:34,-0.002852,7.860000e-10,4.000000e-10,4.996000e-09,0.001521,304.0,4271.476


In [3]:
#adding the elapsed time

timestamps=pd.to_datetime(df["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
df.insert(2,"Elapsed time",runtime)
df.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,injection 1mbar,helium,T-CERNOX
0,NaN,25/10/2022 16:22:05,0.0,-0.002927,7.960000e-10,4.000000e-10,4.996000e-09,0.001554,304.0,4271.476
1,NaN,25/10/2022 16:22:13,8.0,-0.002706,7.960000e-10,4.000000e-10,4.996000e-09,0.001535,303.0,4271.363
2,NaN,25/10/2022 16:22:20,15.0,-0.003150,7.930000e-10,4.000000e-10,4.996000e-09,0.001524,304.0,4271.588
3,NaN,25/10/2022 16:22:27,22.0,-0.002977,7.900000e-10,3.990000e-10,4.996000e-09,0.001534,304.0,4271.251
4,NaN,25/10/2022 16:22:34,29.0,-0.002852,7.860000e-10,4.000000e-10,4.996000e-09,0.001521,304.0,4271.476


In [4]:
df["Live comments"].unique()

array([nan, 'turbo valve closed', 'started H2 injection',
       'closed inj valve', 'inj valve open', 'inj valve closed again'],
      dtype=object)

In [5]:
#Plotting the pressure evolution
plt.figure()
plt.plot((df["Elapsed time"]/3600),df['Barion_1'],marker=".", markersize=5,label='BA1')
plt.plot((df["Elapsed time"]/3600),df['Barion_2'],marker=".", markersize=5,label='BA2')
plt.plot((df["Elapsed time"]/3600),df['injection 1mbar'],marker=".", markersize=5,label='inj 1mbar')

plt.xlabel('Elapsed time (h)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Pressure evolution")
plt.yscale('log')

In [6]:
#TODO
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
T=[]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    

In [7]:
#Plotting the temp evolution
plt.figure()
plt.plot((df["Elapsed time"]/3600),polyfit(A,df["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')

plt.xlabel('Elapsed time (h)')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature evolution")
plt.yscale('linear')

In [8]:
#Annotations on pressure graph in places where a change in the setup was made


In [9]:
#Grid and emission currents, bias currents
#read in the data
hv= pd.read_csv("hivolta log", sep=',', index_col=False)

timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)
hv.head()


,Date,Time,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,VMon8,IMon1,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment
0,25/10/2022,16:32:47.083,0.000,499.88,0.06,0.04,0.06,0.0,0.0,0.02,0.02,0.0206,0.0047,0.0071,0.0044,-0.0004,-0.0104,-0.012,-0.0104,NaN
1,25/10/2022,16:32:48.255,1.172,499.88,0.06,0.04,0.06,0.0,0.0,0.02,0.02,0.0206,0.0047,0.0071,0.0044,-0.0004,-0.0104,-0.012,-0.0104,NaN
2,25/10/2022,16:32:49.446,2.363,499.88,0.06,0.04,0.06,0.0,0.0,0.02,0.02,0.0206,0.0047,0.0071,0.0044,-0.0004,-0.0104,-0.012,-0.0104,NaN
3,25/10/2022,16:32:50.619,3.536,499.88,0.06,0.04,0.06,0.0,0.0,0.02,0.02,0.0206,0.0047,0.0071,0.0044,-0.0004,-0.0104,-0.012,-0.0104,NaN
4,25/10/2022,16:32:51.809,4.726,499.90,0.06,0.04,0.06,0.0,0.0,0.02,0.02,0.0206,0.0047,0.0071,0.0044,-0.0005,-0.0104,-0.012,-0.0104,NaN


In [10]:
#Plotting the different currents vs time 
plt.figure()
plt.plot((hv["Elapsed time"]/3600),hv["IMon1"],marker=".", markersize=5,label='Grid')
plt.plot((hv["Elapsed time"]/3600),hv["IMon2"],marker=".", markersize=5,label='Emission current')
plt.plot((hv["Elapsed time"]/3600),hv["IMon3"],marker=".", markersize=5,label='Bias1')
plt.plot((hv["Elapsed time"]/3600),hv["IMon4"],marker=".", markersize=5,label='Bias2')
plt.plot((hv["Elapsed time"]/3600),hv["IMon5"],marker=".", markersize=5,label='Bias3')

plt.xlabel('Elapsed time (h)')
plt.ylabel('Current (A)')
plt.legend()
plt.title("HiVolta log")
plt.yscale('log')

In [11]:
#remove values up to emission start and negative values
print(hv.iloc[(hv['IMon2'] - 0.2).abs().argsort(),:])
emission=hv.iloc[322:]
emission.describe()

            Date          Time  Elapsed time   VMon1   VMon2   VMon3    VMon4  \
322   25/10/2022  16:39:15.522       388.439    0.12  100.04  999.94  1000.04   
1142  25/10/2022  16:55:37.685      1370.602    0.02  100.06  999.96  1000.12   
1143  25/10/2022  16:55:38.885      1371.802    0.06  100.06  999.96  1000.12   
823   25/10/2022  16:49:15.537       988.454    0.02  100.04  999.96  1000.10   
822   25/10/2022  16:49:14.317       987.234    0.04  100.04  999.96  1000.10   
...          ...           ...           ...     ...     ...     ...      ...   
66    25/10/2022  16:34:06.520        79.437  499.92  100.10  999.02   552.06   
65    25/10/2022  16:34:05.319        78.236  499.92  100.02  922.16   421.66   
62    25/10/2022  16:34:01.748        74.665  499.92   99.98  561.34    60.86   
64    25/10/2022  16:34:04.125        77.042  499.92  100.02  781.92   281.34   
63    25/10/2022  16:34:02.939        75.856  499.92  100.02  691.66   190.94   

       VMon5  VMon6  VMon7 

,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,VMon8,IMon1,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8
count,2035.000000,2035.000000,2035.000000,2035.000000,2035.000000,2035.000000,2035.0,2.035000e+03,2.035000e+03,2035.000000,2035.000000,2035.000000,2035.000000,2035.000000,2035.000000,2035.000000,2035.000000
mean,1606.330111,0.040600,100.049936,999.959292,1000.104619,700.005061,0.0,2.000000e-02,2.000000e-02,-0.031901,0.212469,23.364971,23.396611,23.343429,-0.006046,-0.011033,-0.009695
std,703.638407,0.016021,0.010022,0.009017,0.021033,0.016430,0.0,3.747924e-16,3.747924e-16,0.020591,0.024725,0.307342,0.308524,0.308591,0.002693,0.000820,0.000777
min,388.439000,0.020000,100.040000,999.920000,1000.040000,699.940000,0.0,2.000000e-02,2.000000e-02,-0.093400,-0.284300,22.378700,22.406900,22.354600,-0.010500,-0.012800,-0.011600
25%,997.430500,0.020000,100.040000,999.960000,1000.100000,700.000000,0.0,2.000000e-02,2.000000e-02,-0.046050,0.196500,23.212400,23.242050,23.191050,-0.009400,-0.011700,-0.010100
50%,1606.153000,0.040000,100.040000,999.960000,1000.120000,700.000000,0.0,2.000000e-02,2.000000e-02,-0.025900,0.208300,23.455100,23.489800,23.433400,-0.005200,-0.011000,-0.009700
75%,2215.346000,0.040000,100.060000,999.960000,1000.120000,700.020000,0.0,2.000000e-02,2.000000e-02,-0.015500,0.224700,23.597300,23.627800,23.573650,-0.003500,-0.010400,-0.009200
max,2824.047000,0.120000,100.080000,999.980000,1000.120000,700.020000,0.0,2.000000e-02,2.000000e-02,0.003700,0.301500,23.721200,23.754500,23.705000,-0.002700,-0.009300,-0.007300


In [12]:
#rmv negative values
negs=emission[emission["IMon2"]<0].index
print(negs)
emission=emission.drop(negs)


Int64Index([1912], dtype='int64')


In [13]:
emission.describe()

,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,VMon8,IMon1,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8
count,2034.000000,2034.000000,2034.000000,2034.000000,2034.000000,2034.000000,2034.0,2.034000e+03,2.034000e+03,2034.000000,2034.000000,2034.000000,2034.000000,2034.000000,2034.000000,2034.000000,2034.000000
mean,1605.992718,0.040610,100.049921,999.959282,1000.104612,700.005054,0.0,2.000000e-02,2.000000e-02,-0.031907,0.212713,23.365022,23.396594,23.343346,-0.006046,-0.011033,-0.009694
std,703.646770,0.016018,0.010002,0.009008,0.021035,0.016431,0.0,3.747924e-16,3.747924e-16,0.020594,0.022140,0.307409,0.308599,0.308644,0.002694,0.000820,0.000777
min,388.439000,0.020000,100.040000,999.920000,1000.040000,699.940000,0.0,2.000000e-02,2.000000e-02,-0.093400,0.164800,22.378700,22.406900,22.354600,-0.010500,-0.012800,-0.011600
25%,997.130750,0.020000,100.040000,999.960000,1000.100000,700.000000,0.0,2.000000e-02,2.000000e-02,-0.046075,0.196525,23.212150,23.241975,23.190725,-0.009400,-0.011700,-0.010100
50%,1605.553500,0.040000,100.040000,999.960000,1000.120000,700.000000,0.0,2.000000e-02,2.000000e-02,-0.025950,0.208350,23.455150,23.489800,23.433350,-0.005200,-0.011000,-0.009700
75%,2214.447750,0.040000,100.060000,999.960000,1000.120000,700.020000,0.0,2.000000e-02,2.000000e-02,-0.015500,0.224700,23.597450,23.628050,23.573675,-0.003500,-0.010400,-0.009200
max,2824.047000,0.120000,100.060000,999.980000,1000.120000,700.020000,0.0,2.000000e-02,2.000000e-02,0.003700,0.301500,23.721200,23.754500,23.705000,-0.002700,-0.009300,-0.007300


In [14]:
#Plotting the Emission current stability
plt.figure()
plt.plot(emission["Elapsed time"],emission["IMon2"],marker=".", markersize=5,label='I_em')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (uA)')
plt.legend()
plt.title("Emission current vs time")
plt.yscale('linear')


In [15]:
#TODO figure out what data from HiVolta could be useful
#E.g I_em stability?